# Banco de Chave-Valor

### Objetivo
Definir banco de dados para o sistema salvar sessões de usuários para autenticação rápida.

## Banco de Dados Escolhido
### Redis
O Redis foi selecionado para gerenciar as sessões de usuários devido à sua capacidade de processar dados em milissegundos. Como as informações de autenticação são acessadas a cada clique do usuário, um banco que opera em memória evita latências que comprometeriam a experiência de navegação. Além disso, o Redis oferece vantagens específicas para este cenário:

* **Performance de Ultrabaixa Latência**
    * Por armazenar os dados primariamente na memória RAM, o acesso às informações é quase instantâneo, ideal para validações de tokens em tempo real.
* **Estrutura Simples de Chave-Valor**
    * Foca na eficiência ao associar um identificador único (chave) a um conjunto de dados (valor), eliminando o processamento pesado de consultas relacionais complexas.
* **Gerenciamento de Expiração (TTL)**
    * Possui suporte nativo para configurar um "Tempo de Vida" para os dados, permitindo que sessões inativas expirem e sejam removidas automaticamente do banco.
* **Alta Disponibilidade e Escalabilidade**
    * Suporta a manipulação de milhares de operações por segundo, garantindo que o sistema de autenticação permaneça estável mesmo durante picos massivos de acesso.

### Exemplo de Utilização
Para exemplificar a utilização do Redis, vamos utilizar a biblioteca `fakeredis` que permite **simular localmente** um servidor Redis diretamente no Python, **sem necessidade de instalar ou subir o Redis real**.

In [1]:
# Instalando o fakeredis
# %pip install fakeredis

In [2]:
# Importando as bibliotecas
import fakeredis
import json
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Criando a conexão simulada
r = fakeredis.FakeRedis(decode_responses=True) # O decode_responses=True permite trabalhar com strings em vez de bytes

def criar_sessao(user_id, token, expira_em_segundos=30):
    """Simula o login de um usuário e cria uma sessão com TTL"""
    chave = f"session:user:{user_id}"
    dados_sessao = {
        "user_id": user_id,
        "token": token,
        "login_at": time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Armazena como string JSON e define o tempo de expiração (TTL)
    r.setex(chave, expira_em_segundos, json.dumps(dados_sessao))
    print(f"Sessão criada para o usuário {user_id}. Expira em {expira_em_segundos}s.")

def validar_sessao(user_id):
    """Busca a sessão no Redis de forma instantânea"""
    chave = f"session:user:{user_id}"
    sessao = r.get(chave)
    
    if sessao:
        print(f"Acesso autorizado! Dados da sessão: {sessao}")
        return json.loads(sessao)
    else:
        print("Acesso negado: Sessão expirada ou inexistente.")
        return None

### Simulação Completa

In [4]:
# Simulação de Login
criar_sessao(user_id=12345, token="jwt_token_exemplo_abc123", expira_em_segundos=2)

# Validação Instantânea (Sucesso)
print("\nVerificando sessão imediatamente...")
validar_sessao(12345)

# Simula Espera para Expiração
print("\nAguardando 3 segundos para a sessão expirar...")
time.sleep(3)

# Validação após expiração (Falha)
validar_sessao(12345)

Sessão criada para o usuário 12345. Expira em 2s.

Verificando sessão imediatamente...
Acesso autorizado! Dados da sessão: {"user_id": 12345, "token": "jwt_token_exemplo_abc123", "login_at": "2026-02-20 09:44:31"}

Aguardando 3 segundos para a sessão expirar...
Acesso negado: Sessão expirada ou inexistente.
